#  Converting a CSV file to LAS in python

LAS is an industry standard binary data format for 3D point cloud data collected using airborne LiDAR and terrestrial laser scanner (TLS). Most recent specification is [LAS 1.4 R15](http://www.asprs.org/wp-content/uploads/2019/07/LAS_1_4_r15.pdf). Various lossless compressed formats also exists such as LAZ and zLAS. See [here](https://www.loc.gov/preservation/digital/formats/fdd/fdd000418.shtml) for more details.

A LAS file contains following sections:
1. Public header block
2. Variable Length Records (VLRs) (optional)
3. Point Data Records (PDRs)
4. Extended Variable Length Records (EVLRs) (optional)

We will use [LASpy python library](https://github.com/laspy/laspy) to write a LIDAR data in LAS format.


Download this [csv file](https://daacingest.ornl.gov/ingest/browse/095355e2b8/AK_fte_TLS_plot1.csv). We will convert this to LAS file.

Let's open and check the first few lines of the csv file. We will use python's pandas module

In [100]:
import pandas as pd
# incsv = pd.read_csv("AK_fte_TLS_plot1.csv", nrows=10)
incsv = pd.read_csv("AK_fte_TLS_plot1.csv")
incsv.head()

,x,y,z
0,386286.201360,7.547559e+06,802.495770
1,386466.839783,7.547565e+06,736.517112
2,386465.316100,7.547565e+06,736.543581
3,386465.804590,7.547565e+06,737.190824
4,386465.977808,7.547565e+06,737.040793


The file contains the x, y, z coordinates for each point. From [metadata information file](https://daacingest.ornl.gov/ingest/browse/095355e2b8/AK_TLS_data_documentation.txt), we know that the units of these are in meters and projection is `UTM zone 5N`. Now, we will use laspy to convert the csv to a properly formatted las file.

In [102]:
import laspy
import numpy as np

# set LAS version to 1.4
lasheader = laspy.header.Header(file_version=1.4)
olas = laspy.file.File("AK_fte_TLS_plot1.las", mode="w", header=lasheader)

xoffset = np.floor(incsv.x.min())
yoffset = np.floor(incsv.y.min())
zoffset = np.floor(incsv.z.min())

# because we have six decimals
x_scale = y_scale = z_scale = 0.000001


olas.header.offset = [xmin,ymin,zmin]
olas.header.scale = [x_scale,y_scale,z_scale]

olas.X = (incsv.x.values - xoffset)/x_scale
olas.Y = (incsv.y.values - zoffset)/y_scale
olas.Z = (incsv.z.values - zoffset)/z_scale

olas.close()

Now let's read the las file we just created, print its header and check everything is ok.

In [103]:
inlas = laspy.file.File("AK_fte_TLS_plot1.las", mode="r")
print("LAS Version: " + str(inlas.header.version))
print("Date: " + str(inlas.header.date))
print("Point Format: " + str(inlas.header.data_format_id))
print("Number of Point Records: " + str(len(inlas)))
print("Min X Y Z: " + str(inlas.header.min))
print("Max X Y Z: " + str(inlas.header.max))
print("Offset X Y Z: " + str(inlas.header.offset))
print("Scale X Y Z: " + str(inlas.header.scale))
print("Point Dimensions: ")
for dim in inlas.point_format:
    print("    " + dim.name)

print("Data Array: ")
data = np.vstack((inlas.x, inlas.y, inlas.z)).transpose()  
print("X", inlas.x)
print("Y", inlas.y)
print("Z", inlas.z)
inlas.close()

LAS Version: 1.4
Date: 2020-06-02 00:00:00
Point Format: 0
Number of Point Records: 50466852
Min X Y Z: [386286.741259, 7545410.516352, 736.292717]
Max X Y Z: [386694.773087, 7545410.516352, 841.045955]
Offset X Y Z: [386286.0, 7547558.0, 736.0]
Scale X Y Z: [1e-06, 1e-06, 1e-06]
Point Dimensions: 
    X
    Y
    Z
    intensity
    flag_byte
    raw_classification
    scan_angle_rank
    user_data
    pt_src_id
Data Array: 
X [386368.20136  386548.839783 386547.316099 ... 386545.562493 386573.582644
 386559.288507]
Y [7545410.516352 7545410.516352 7545410.516352 ... 7545410.516352
 7545410.516352 7545410.516352]
Z [815.49577  749.517111 749.54358  ... 748.56075  752.514111 750.368455]


You can now view the las files in a las viewer to check if they look ok. Here is one good/free one: http://lidarview.com/